# Notes

**Question**: *What's TF Model Garden.*

**Aha**: A lib that allows you acess pre-build models, to adjust to your necessities.

**URL**: https://www.tensorflow.org/tfmodels/nlp/fine_tune_bert

# Code

In [1]:
%%capture

!pip install -q opencv-python # why i need this?
!pip install tensorflow-text
!pip install -q tf-models-official

## Setup

### Lib

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_models as tfm
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

### Resources

In [3]:
## this is how
## we acess
## Bert model

gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

## Load and preprocess the dataset

### Get the dataset from TensorFlow Datasets

In [4]:
batch_size=32
glue, info = tfds.load('glue/mrpc',
                       with_info=True,
                       batch_size=batch_size)

Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/mrpc/2.0.0. Subsequent calls will reuse this data.


In [5]:
print(info.features)
print('\n')
print("Classes:")
print(info.features['label'].names)

FeaturesDict({
    'idx': int32,
    'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    'sentence1': Text(shape=(), dtype=string),
    'sentence2': Text(shape=(), dtype=string),
})


Classes:
['not_equivalent', 'equivalent']


In [6]:
example_batch = next(iter(glue['train']))

for key, value in example_batch.items():
  print(f"{key:9s}: {value[0].numpy()}")

idx      : 1680
label    : 0
sentence1: b'The identical rovers will act as robotic geologists , searching for evidence of past water .'
sentence2: b'The rovers act as robotic geologists , moving on six wheels .'


### Process Data

In [7]:
tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
    lower_case=True)

In [8]:
tokens = tokenizer(tf.constant(["TensorFlow!"]))
tokens

<tf.RaggedTensor [[[23435, 12314], [999]]]>

In [9]:
special = tokenizer.get_special_tokens_dict()
special

{'vocab_size': 30522,
 'start_of_sequence_id': 101,
 'end_of_segment_id': 102,
 'padding_id': 0,
 'mask_id': 103}

In [10]:
max_seq_length = 64

packer = tfm.nlp.layers.BertPackInputs(
    seq_length=max_seq_length,
    special_tokens_dict = tokenizer.get_special_tokens_dict())

In [11]:
sentences1 = ["hello tensorflow"]
tok1 = tokenizer(sentences1)
tok1

<tf.RaggedTensor [[[7592], [23435, 12314]]]>

In [12]:
sentences2 = ["goodbye tensorflow"]
tok2 = tokenizer(sentences2)
tok2

<tf.RaggedTensor [[[9119], [23435, 12314]]]>

In [13]:
packed = packer([tok1, tok2])

for key, tensor in packed.items():
  print(f"{key:15s}: {tensor[:, :12]}")

input_word_ids : [[  101  7592 23435 12314   102  9119 23435 12314   102     0     0     0]]
input_mask     : [[1 1 1 1 1 1 1 1 1 0 0 0]]
input_type_ids : [[0 0 0 0 0 1 1 1 1 0 0 0]]


Put All Together

In [14]:
class BertInputProcessor(tf.keras.layers.Layer):
  def __init__(self, tokenizer, packer):
    super().__init__()
    self.tokenizer = tokenizer
    self.packer = packer

  def call(self, inputs):
    tok1 = self.tokenizer(inputs['sentence1'])
    tok2 = self.tokenizer(inputs['sentence2'])

    packed = self.packer([tok1, tok2])

    if 'label' in inputs:
      return packed, inputs['label']
    else:
      return packed

Apply to the dataset

In [15]:
bert_inputs_processor = BertInputProcessor(tokenizer, packer)

In [16]:
glue_train = glue['train'].map(bert_inputs_processor).prefetch(1)

In [17]:
example_inputs, example_labels = next(iter(glue_train))

In [18]:
for key, value in example_inputs.items():
  print(f'{key:15s} shape: {value.shape}')

print(f'{"labels":15s} shape: {example_labels.shape}')

input_word_ids  shape: (32, 64)
input_mask      shape: (32, 64)
input_type_ids  shape: (32, 64)
labels          shape: (32,)


In [19]:
plt.pcolormesh(example_inputs['input_word_ids'])

In [20]:
plt.pcolormesh(example_inputs['input_mask'])

In [21]:
plt.pcolormesh(example_inputs['input_type_ids'])

In [22]:
glue_validation = glue['validation'].map(bert_inputs_processor).prefetch(1)
glue_test = glue['test'].map(bert_inputs_processor).prefetch(1)

## Build, train and export the model

### Build the model

In [23]:
import json

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())
config_dict

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [24]:
encoder_config = tfm.nlp.encoders.EncoderConfig({
    'type':'bert',
    'bert': config_dict
})

In [25]:
bert_encoder = tfm.nlp.encoders.build_encoder(encoder_config)
bert_encoder

In [26]:
bert_classifier = tfm.nlp.models.BertClassifier(network=bert_encoder, num_classes=2, )

In [27]:
bert_classifier(example_inputs, training=True).numpy()[:10]

array([[ 0.4472508 , -0.8242993 ],
       [-0.12412836, -0.32979816],
       [ 0.65828896, -0.4023409 ],
       [ 0.29047823, -0.25196293],
       [-0.03875865, -0.06445092],
       [ 1.0850021 , -0.39459246],
       [-0.12364703, -0.5967975 ],
       [ 0.25581485, -0.6748352 ],
       [ 0.24329147, -0.37364754],
       [ 0.48788297, -0.58187354]], dtype=float32)

In [28]:
tf.keras.utils.plot_model(bert_encoder, show_shapes=True, dpi=1)

### Restore Weights

In [29]:
checkpoint = tf.train.Checkpoint(encoder=bert_encoder)
checkpoint.read(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()

### Set up and Optimazer

In [30]:
# Set up epochs and steps
epochs = 2
batch_size = 32
eval_batch_size = 32

train_data_size = info.splits['train'].num_examples
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(0.1 * num_train_steps)
initial_learning_rate=2e-5

In [31]:
linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=0,
    decay_steps=num_train_steps)

In [32]:
warmup_schedule = tfm.optimization.LinearWarmup(
    warmup_learning_rate = 0,
    after_warmup_lr_sched = 0,
    warmup_steps = warmup_steps
)

In [33]:
x = tf.linspace(0, num_train_steps, 1001)
y = [warmup_schedule(xi) for xi in x]
plt.plot(x,y)
plt.xlabel('Train step')
plt.ylabel('Learning rate')

Text(47.097222222222214, 0.5, 'Learning rate')

In [34]:
optimizer = tf.keras.optimizers.AdamW(
    learning_rate = warmup_schedule)

In [35]:
#metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
#loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
bert_classifier.compile(optimizer="AdamW", loss='SparseCategoricalCrossentropy', metrics='accuracy')

In [36]:
bert_classifier.evaluate(glue_validation)

13/13 [==============================] - 7s 127ms/step - loss: 5.0962 - accuracy: 0.6838


[5.09616231918335, 0.6838235259056091]

In [37]:
bert_classifier.fit(
      glue_train,
      validation_data=(glue_validation),
      batch_size=32,
      epochs=epochs)

Epoch 1/2
115/115 [==============================] - 84s 406ms/step - loss: 0.8229 - accuracy: 0.3702 - val_loss: 0.6931 - val_accuracy: 0.3162
Epoch 2/2
115/115 [==============================] - 47s 406ms/step - loss: 0.6931 - accuracy: 0.3255 - val_loss: 0.6931 - val_accuracy: 0.3162


In [89]:
my_examples = {
        'sentence1':[
            'The rain in Spain falls mainly on the plain.',
            'Look I fine tuned BERT.'],
        'sentence2':[
            'It mostly rains on the flat lands of Spain.',
            'Is it working? This does not match.']
    }

In [90]:
my_examples

{'sentence1': ['The rain in Spain falls mainly on the plain.',
  'Look I fine tuned BERT.'],
 'sentence2': ['It mostly rains on the flat lands of Spain.',
  'Is it working? This does not match.']}

In [96]:


library(keras3)
bert_inputs_processor = BertInputProcessor(tokenizer, packer)
ex_packed = bert_inputs_processor(my_examples)

/bin/bash: -c: line 1: syntax error near unexpected token `"keras"'
/bin/bash: -c: line 1: `remove.packages("keras")'
/bin/bash: -c: line 1: syntax error near unexpected token `"keras3"'
/bin/bash: -c: line 1: `install.packages("keras3") # or remotes::install_github("rstudio/keras")'


NameError: name 'library' is not defined

In [ ]:
my_logits = bert_classifier(ex_packed, training=False)

result_cls_ids = tf.argmax(my_logits)
result_cls_ids